# Policy generation

In [1]:
# Imports

from __future__ import print_function

import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook

import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

## Hyperparameters

In [8]:
# Hyperparameters
batch_size = 1000   # num of training examples to train on in a batch
limit_timesteps = 15 # sequence length
num_hidden_layers = 51
state_dim = 3 # dimensions of the state variable passed as input to the LSTM model
future = 10

## Define and load PyTorch RNN Model

In [13]:
# Sequence model
# model used to train RNN with PyTorch
class Sequence(nn.Module):
    def __init__(self, state_dim, num_hidden_layers):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(state_dim, num_hidden_layers)
        self.lstm2 = nn.LSTMCell(num_hidden_layers, num_hidden_layers)
        self.linear = nn.Linear(num_hidden_layers, state_dim)

    def forward(self, input, future = 0):
        outputs = []
        
        h_t = Variable(torch.zeros(input.size(0), num_hidden_layers).double(), requires_grad=False)
        c_t = Variable(torch.zeros(input.size(0), num_hidden_layers).double(), requires_grad=False)
        h_t2 = Variable(torch.zeros(input.size(0), num_hidden_layers).double(), requires_grad=False)
        c_t2 = Variable(torch.zeros(input.size(0), num_hidden_layers).double(), requires_grad=False)
        
        # run 19 times
        # input_t.size = 5 x 5
        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            input_t = input_t.squeeze(1)
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            if (future == 0):
                outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        print(outputs.size())
        return outputs

In [6]:
# Load model only
seq = Sequence(state_dim, num_hidden_layers)
params = torch.load('ACAS_theparameters.pt')
seq.load_state_dict(params)
seq.double()

Sequence (
  (lstm1): LSTMCell(3, 51)
  (lstm2): LSTMCell(51, 51)
  (linear): Linear (51 -> 3)
)

## The cells below are used for generating a policy for an NMAC

### Helper functions

Declaring constants and writing helper functions to detect NMACs

In [11]:
import math
import geopy
import geopy.distance

def is_NMAC(point_1, point_2):
    '''    
    Parameters:
        - point_1 -> list in order of [lat,long,alt]
        - point_2 -> list in order of [lat,long,alt]
    Returns:
        Whether or not two planes are in an NMAC
    
    An NMAC between two planes in 3d space is specified by less than 200 ft (60.96m) of vertical separation
    or less than 500 ft (152.4 m) of horizontal seperation.
    '''    
    p1 = geopy.point.Point(point_1)
    p2 = geopy.point.Point(point_2)
    
    if (abs(point_1[2] - point_2[2]) <= 60.69
           or geopy.distance.vincenty(p1, p2).m <= 152.4):
        return True
    
    return False

def unnormalize(path):
    path *= np.load("model_stdevs.npy")
    path += np.load("model_means.npy")
    
    return path
    
# constant variables
future = 10
ASCEND = 1500 / 60
DESCEND = - 1500 / 60

### Output policy

Pseudocode in Python that we will convert into Julia to be run on simulator on VM.

Takes input for both plane1 and plane 2, runs our LSTM model on the plane's data points, generates the future timesteps for both planes. Then outputs a policy/resolution advisory.

Simple avoidance policy:
- if both planes are in NMAC
    - plane1 is above plane2
        - plane2 descend
    - plane2 is above or equivalent to plane1
        - plane 2 ascend

In [12]:
# assuming plane1_input is 10 previous timesteps including the current state 

# step 1.) clean input. drop irrelevant columns. format plane1_input to just be x_disp, y_disp, geoaltitude

# step 2.) make a prediction for both plane's trajectory 
test_pred1 = seq(plane1_input,future).data.squeeze(1).numpy()
test_pred1_latlon = test_pred1[:, :, -3:]

test_pred2 = seq(plane2_input,future).data.squeeze(1).numpy()
test_pred2_latlon = test_pred2[:, :, -3:]

# format plane 1 predictions
latlon = plane1_input.data.numpy()[:, -3:]
latlon = np.insert(latlon, latlon.shape[0], test_pred1_latlon[0], axis=0)
path1 = unnormalize(latlon.cumsum(axis=0)) # cumsum to stack displacements for further timesteps, and unnormalize data
path1x, path1y, path1z = np.split(path1, 3, axis=1)
path1z = path1z.flatten()

# find corresponding last row of input data (last row of input data would be current spot of plane)
plane1_currstate = plane1_input[-1]
# add plane 1's current lat lon with its x and y displacement to get its next trajectory point
path1x = path1x + plane1_currstate[0] # assuming 0 is lat column
path1y = path1y + plane1_currstate[1] # assuming 1 is lon column

# plotting for visualization
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot(path1x[0], path1y[0], path1z[0], 'b+')
ax.plot(path1x[:-future], path1y[:-future], path1z[:-future], 'b.')
for i in range(1,future+1):
    plt.plot(path1x[-i], path1y[-i], path1z[-i], 'ro')

    
# repeat same process but for plane 2
# format plane 2 predictions
latlon = plane2_input.data.numpy()[:, -3:]
latlon = np.insert(latlon, latlon.shape[0], test_pred2_latlon[0], axis=0)
path2 = unnormalize(latlon.cumsum(axis=0)) # cumsum to stack displacements for further timesteps, and unnormalize data
path2x, path2y, path2z = np.split(path2, 3, axis=1)
path2z = path2z.flatten()

# find corresponding last row of input data (last row of input data would be current spot of plane)
plane2_currstate = plane2_input[-1]
# add plane 2's current lat lon with its x and y displacement to get its next trajectory point
path2x = path2x + plane2_currstate[0] # assuming 0 is lat column
path2y = path2y + plane2_currstate[1] # assuming 1 is lon column

# plotting for visualization
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot(path2x[0], path2y[0], path2z[0], 'b+')
ax.plot(path2x[:-future], path2y[:-future], path2z[:-future], 'b.')
for i in range(1,future+1):
    plt.plot(path2x[-i], path2y[-i], path2z[-i], 'ro')
plt.show()


# predict NMAC and generate a policy for the 2 planes
for i in range(1,future+1):
    point_1 = [path1x[-i], path1y[-i], path1z[-i]]
    point_2 = [path2x[-i], path2y[-i], path2z[-i]]
    
    if (is_NMAC(point_1, point_2)):
        if (point1z[-i] > point2z[-i]):
            resolution_advisory = ASCEND
        else:
            resolution_advisory = DESCEND

return resolution_advisory
            
    

NameError: name 'plane1_input' is not defined